# Steering Wheel Experiments
## Imports

In [ ]:
from pathlib import Path
from datetime import date
import os
import subprocess
from datetime import datetime
from copy import deepcopy
from revcan.reverse_engineering.models.experiment import Experiment

## Create Necessary Experiment Files

### TODO: 
Set & check the following parameters:
- **Car Model File Name**: Specify the name of the car model file (`{model}_{vin}_{date-created}.json`).
- **Car Model Directory**: Check the directory path where the car model file is stored (Default: `../../data/car_metadata`).
- **Experiment Folder**: Set the path for the experiment folder where the results will be saved (Default: `../../data/experiments/{car}/{experiment}/{date-today}`).

In [ ]:
car_model_file_name = "model_vin_date-created.json" # TODO: Set correct car model file name
car_model_dir = "../../data/car_metadata" # TODO: Check directory of car model file
experiment_folder = "../../data/experiments/car/steering_wheel/" + date.today().strftime("%Y-%m-%d") # TODO: Set directory where experiment files should be saved to
angles=["0","90L","180L","360L","90R","180R","360R"]
car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)


Path(experiment_folder).mkdir(parents=True, exist_ok=True)

angles=["0","90L","180L","360L","90R","180R","360R"]

for angle in angles:

    experiment_file_path = os.path.join(experiment_folder, angle+".json")
    experiment_name = f"steering wheel {angle} deg"
    experiment_description = "steering wheel signal discovery using 7 positions."
    signal_selection ="all"
    print(f"Experiment file path: {experiment_file_path}")
    !python ../scripts_for_doip_new/04_create_experiment.py --car_model_file_path '{car_model_file_path}' --experiment_file_path '{experiment_file_path}' --experiment_name '{experiment_name}' --experiment_description '{experiment_description}' --signal_selection {signal_selection}




## [Optional]: Set Experiment Names manually

In [ ]:
car_model_file_name = "model_vin_date-created.json" # TODO: Set correct car model file name
car_model_dir = "../../data/car_metadata" # TODO: Check directory of car model file
experiment_folder = "../../data/experiments/car/steering_wheel/" + date.today().strftime("%Y-%m-%d") # TODO: Set directory where experiment files should be saved to
angles=["0","90L","180L","360L","90R","180R","360R"]
car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)

# Measurements

## Measurement - Steering wheel angle
**Before** executing the next Cell:

- Set steering wheel to required angle

Start measurement


In order to apply **pre-filtering** strategy:

After **2** measurements: 

- execute the Filter Cells below and **return**
- complete the remaining measurements.
- Continue normal flow of cells.

## Measurement 0°
Before executing the next Cell:

Bring the steering wheel into the neutral position (0°)


In [ ]:
!python ../scripts_for_doip_new/05_read_data_parallel_2.py --experiment_file_path '{os.path.join(experiment_folder, "0"+".json")}' --activate_logging false --num_samples 5


## Measurement 90° left
Before executing the next Cell:

Bring the steering wheel into the 90° left position 


In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "90L"+".json")}' --activate_logging true --num_samples 5

## Measurement 180° left
Before executing the next Cell:

Bring the steering wheel into the 180° left position 

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "180L"+".json")}' --activate_logging true --num_samples 5

## Measurement 360° left
Before executing the next Cell:

Bring the steering wheel into the 360° left position 

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "360L"+".json")}' --activate_logging true --num_samples 5

## Measurement 90° right
Before executing the next Cell:

Bring the steering wheel into the 90° right position 

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "90R"+".json")}' --activate_logging true --num_samples 5

## Measurement 180° right
Before executing the next Cell:

Bring the steering wheel into the 180° right position 

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "180R"+".json")}' --activate_logging true --num_samples 5

## Measurement 360° right
Before executing the next Cell:

Bring the steering wheel into the 360° right position 

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "360R"+".json")}' --activate_logging true --num_samples 5

# Evaluation

## [Optional] Display Experiment Metadata

In [ ]:
# Display Experiment
for angle in angles:
    experiment_file = os.path.join(experiment_folder, f"{angle}.json")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"


## Load Experiments

Load all relevant experiments needed for this analysis

In [6]:
experiment_files = {}
experiments = {}

for angle in angles:
    experiment_files[angle] = os.path.join(experiment_folder, f"{angle}.json")
    experiments[angle] = Experiment.load(experiment_files[angle])
pass


## Add Ground truth
Add ground truth data for steering wheel angle. By experiment design these are fixed values, and are added manually in this case

In [ ]:
from datetime import timedelta
import datetime
from typing import List
from copy import deepcopy
from revcan.reverse_engineering.models.experiment import Extern_Signal, Value

angles_to_value_map={"0":0,"90L":90,"180L":180,"360L":360,"90R":-90,"180R":-180,"360R":-360}


for angle in angles:
    experiments[angle].external_measurements.append(Extern_Signal( name = "Angle", id = 1, values = []))
    values:List[Value] = []
    for value in experiments[angle].measurements[0].values:
        values.append(Value(value= [angles_to_value_map[angle]], time= value.time))
    experiments[angle].external_measurements[0].values = deepcopy(values)
    print(f"Experiment Steering Wheel angle {angle}°: {len(experiments[angle].external_measurements[0].values)} ground truth values")


## Filter

Concat all experiments

In [ ]:
combined_experiment = experiments[angles[0]] 
combined_experiment.name = f"Non constant singals for angle experiment"
combined_experiment.description = f"Non constant signals for angle experiment with {len(angles)} different angles."

for angle in angles[1:]:
    combined_experiment.experiment_runtime_seconds += experiments[angle].experiment_runtime_seconds
    for measurement in experiments[angle].measurements:
        for x in combined_experiment.measurements:
            if x.serverid == measurement.serverid:
                if x.did == measurement.did:
                    x.values.extend(measurement.values)
                    break
  


Keep only non constant signals

In [ ]:
number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements before filtering", )
print("Applaying remove constant values filter")

combined_experiment.keep_non_constant_signals(combined_experiment)

number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements after filtering", )

Remove all previously found constant signals from individual experiments

In [ ]:
for angle in angles:
    experiments[angle].keep_signals_by_list(signals_list=combined_experiment.measurements)
    experiment_file = os.path.join(experiment_folder, f"{angle}.json")
    experiments[angle].save(f"{experiment_file}")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"

Save combined experiment file

In [ ]:
combined_experiment_file = os.path.join(experiment_folder, f"combined.json")
combined_experiment.save(f"{combined_experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{combined_experiment_file}"

### End of Experiment 

Continue with Linear evaluation